# Output Parser
**Output Parser**는 대규모 언어 모델(LLM, Large Language Model)의 출력 결과를 **애플리케이션에서 활용할 수 있도록 적절한 형식으로 변환**하는 도구이다.
- LLM은 일반적으로 텍스트 형태로 응답을 생성하지만, 이 텍스트는 그대로 활용하기 어려운 경우가 많다.
- Output Parser는 이러한 **비구조적 텍스트 데이터를 구조화된 데이터로 변환**하여 프로그램에서 활용 가능하도록 만든다.
- 예를 들어, 키워드 리스트를 뽑거나 JSON 형식으로 정보를 변환하는 데 사용된다.

## 주요 Output Parser 종류

1. **CommaSeparatedListOutputParser**
   - 쉼표로 구분된 텍스트를 파싱하여 리스트 형태로 변환한다.
   - 예: `"사과, 바나나, 포도"` → `["사과", "바나나", "포도"]`
2. **JsonOutputParser**
   - LLM의 출력이 JSON 형식일 때 이를 Python의 `dict` 객체로 변환한다.
   - JSON(JavaScript Object Notation)은 데이터 구조를 표현하기 위한 경량 포맷이다.
3. **PydanticOutputParser**
   - JSON 데이터를 Python의 [Pydantic](https://docs.pydantic.dev) 모델로 변환한다.
   - Pydantic은 데이터 유효성 검사와 설정 관리에 널리 사용되는 Python 라이브러리이다.
4. **StrOutputParser**
   - 모델의 출력 결과를 단순 문자열로 반환한다.
   - Chat 기반 모델은 Message 객체의 속성으로 LLM 결과를 반환한다. 거기에서 응답 문자열만 추출해서 반환한다.
> `JsonOutputParser`, `PydanticOutputParser` 는 모두 Pydantic을 사용해 데이터 구조(schema)를 정의하고, 해당 구조에 따라 출력을 검증하고 변환한다.

## 주요 메소드
- `parse(text: str)`
  - LLM이 생성한 문자열 응답을 받아 정해진 구조로 변환하여 반환한다.
- `get_format_instructions() -> str`
  - 각 OutputParer가 변환할 수있는 형식으로 LLM이 응답하도록 하는 프롬프트 텍스트를 반환한다.
  - 이 내용을 프롬프트에 넣어서 LLM이 정확한 포맷으로 응답하도록 유도한다.
  
## 참고
- Output Parser는 일반적으로 [`Runnable`](05_chaing_LECL.ipynb#Runnable) 인터페이스를 상속하여 구현되며, `invoke()` 메서드를 통해 실행할 수 있다.
- `invoke()`는 내부적으로 `parse()`를 호출하여 동작한다.
- 필요한 경우 Output Parser를 직접 구현하여 사용자 정의 출력 포맷을 처리할 수도 있다. 


## StrOutputParser
- 모델(LLM)의 출력 결과를 string으로 변환하여 반환하는 output parser.
- Chat Model은  Message 객체에서 content 속성값을 추출하여 문자열로 반환한다.

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


# 단순 질의용 prompt 생성-from_template() 메송드 이용. (PromptTemplate용)
prompt = ChatPromptTemplate.from_template(
    template="한국의 {topic}에 관련된 속담 {count}개를 알려줘. 목록 형식으로 출력해줘."
)
model = ChatOpenAI(model="gpt-5-mini")
parser = StrOutputParser()
#prompt -> model-> parser

query = prompt.invoke({"topic":"호랑이","count":3})
res = model.invoke(query)
final_res = parser.invoke(res)

In [ ]:
res

AIMessage(content='1. 호랑이 굴에 들어가야 호랑이 새끼를 잡는다. — 큰 성과를 얻으려면 큰 위험을 감수해야 한다는 뜻.  \n2. 호랑이에게 물려가도 정신만 차리면 산다. — 위급한 상황에서도 침착하면 살아남을 수 있다는 뜻.  \n3. 호랑이 없는 골에 여우가 왕. — 강자가 없으면 약한 자가 득세한다는 뜻.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1137, 'prompt_tokens': 30, 'total_tokens': 1167, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 1024, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CnbHuWBh5H1IeYzbU66baH8wohlqf', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b2a12-ec74-78d1-a203-85f9154df59a-0', usage_metadata={'input_tokens': 30, 'output_tokens': 1137, 'total_tokens': 1167, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 1024}})

In [5]:
final_res

'1. 호랑이 굴에 들어가야 호랑이 새끼를 잡는다. — 큰 성과를 얻으려면 큰 위험을 감수해야 한다는 뜻.  \n2. 호랑이에게 물려가도 정신만 차리면 산다. — 위급한 상황에서도 침착하면 살아남을 수 있다는 뜻.  \n3. 호랑이 없는 골에 여우가 왕. — 강자가 없으면 약한 자가 득세한다는 뜻.'

## CommaSeparatedListOutputParser

- 쉼표로 구분된 텍스트를 파싱하여 리스트 형태로 변환한다.
  - "a,b,c" => ['a','b','c']

In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()
txt = "이순신,유관순,안중근,강감찬,세종대왕"
txt = "요청하신 이름은 이순신과 유관순과 안중근입니다."
r1 = parser.parse(txt)
r2 = parser.invoke(txt)
print(type(r1), type(r2))

<class 'list'> <class 'list'>


In [ ]:
print(r1)
print(r2)

['이순신', '유관순', '안중근', '강감찬', '세종대왕']


In [12]:
# 이 output parser에 맞는 출력을 모델이 하도록 하는 지시문으로 프롬프트에 추가한다.
parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [15]:
parser = CommaSeparatedListOutputParser()
prompt = ChatPromptTemplate(
    messages=[
        {"role":"system", "content":"Output Format:{format_instruction}"},
        {"role":"user", "content":"{subject}에 대해 다섯가지를 나열해 주세요."}
    ],
    partial_variables = {"format_instruction":parser.get_format_instructions()}
)
# partial_variables={"input_variable":넣을 값,} 
# => input_variable 의 값을 prompt Template을 만들면서 넣어준다. (invoke 시 넣는 것이 아니라 생성시 설정)

model = ChatOpenAI(model="gpt-5-mini")

query = prompt.invoke({"subject":"자동차 종류"})
res = model.invoke(query)

In [17]:
res.content
final_res = parser.invoke(res)
final_res

['세단', 'SUV', '해치백', '쿠페', '컨버터블']

In [18]:
# 질의 -> (Prompt) -> query -> (Model) -> res -> (Parser) -> 최종답변
# Chain: 위 실행흐름을 자동화(Pipeline)
chain = prompt | model | parser
res = chain.invoke({"subject":"오픈소스 LLM 모델 이름"})
print(res)

['LLaMA 2', 'Falcon 40B', 'Mistral 7B', 'BLOOM', 'GPT-J-6B']


## JsonOutputParser

- JSON 형식의 응답을 dictionary로 반환한다.
- JSON 형식을 정하려는 경우 [Pydantic](Ref_typing_Pydantic.ipynb)을 이용해 JSON 스키마를 정의하여 JsonOutputParser 생성시 전달한다.
  - Pydantic 모델클래스를 이용해 LLM 모델이 응답할 때 json의 어떤 key에 어떤 응답을 작성할 지 Field로 정의한다.
  - Schema 지정은 필수는 아니다. 
- LLM이 JSON Schema를 따르는 형태로 응답을 하면 JsonOutputParser는 Dictionary로 변환한다.

In [20]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()
parser.get_format_instructions()

txt = '{"name":"홍길동","age":20}'
r = parser.invoke(txt)
print(type(r))
r

<class 'dict'>


{'name': '홍길동', 'age': 20}

In [21]:
r['age']

20

In [28]:
# Json 응답 스키마(구조설계) 정의 - > pydantic
from pydantic import BaseModel, Field

class PersonInfo(BaseModel):
    name:str = Field(description="조회한 사람의 이름")
    yob: int = Field(description="조회한 사람이 태어난 년도")
    yod: int = Field(description="조회한 사람이 사망한 년도")
    profile: str = Field(description="조회한 사람의 주요 업적")

# 변수 : key
# Field.description : 변수에 넣을 내용

In [30]:
parser2 = JsonOutputParser(pydantic_object=PersonInfo)
print(parser2.get_format_instructions())

STRICT OUTPUT FORMAT:
- Return only the JSON value that conforms to the schema. Do not include any additional text, explanations, headings, or separators.
- Do not wrap the JSON in Markdown or code fences (no ``` or ```json).
- Do not prepend or append any text (e.g., do not write "Here is the JSON:").
- The response must be a single top-level JSON value exactly as required by the schema (object/array/etc.), with no trailing commas or comments.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]} the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema (shown in a code block for readability only — do not include any backticks or Markdown in your output):


In [35]:
# parser = JsonOutputParser()
parser = JsonOutputParser(pydantic_object=PersonInfo)
prompt = ChatPromptTemplate(
    [
        {"role":"system", "content":"Output Format: {output_format}"},
        {"role":"user", "content":"{query}"}
    ],
    partial_variables={"output_format":parser.get_format_instructions()}
)
model = ChatOpenAI(model="gpt-5-mini")

query = prompt.invoke({"query":"이순신 장군에 대해서 알려줘."})
res = model.invoke(query)

In [36]:
final_res = parser.invoke(res)

In [37]:
print(type(res.content), type(final_res))
final_res

<class 'str'> <class 'dict'>


{'name': '이순신',
 'yob': 1545,
 'yod': 1598,
 'profile': "조선의 해군 장군. 임진왜란(1592-1598) 동안 조선 수군을 이끌어 한산도 대첩(1592), 명량 해전(1597) 등에서 결정적인 승리를 거두어 해상 교통로를 지키고 왜군의 남하를 저지했다. 거북선 운용으로 유명하며 난중일기라는 전쟁 일기를 남겼다. 1598년 노량해전에서 전사했으며 '충무공'으로 존경받는다."}

## PydanticOutputParser

- JSON 형태로 받은 응답을 Pydantic 모델로 변환하여 반환한다.
- 구현은 JsonOutputParser와 동일한데 parsing 결과를 pydantic 모델타입으로 반환한다.

In [44]:
from pydantic import BaseModel, Field

class PersonInfo(BaseModel):
    name:str = Field(description="조회한 사람의 이름")
    yob: int = Field(description="조회한 사람이 태어난 년도")
    yod: int = Field(description="조회한 사람이 사망한 년도")
    profile: str = Field(description="조회한 사람의 주요 업적")


In [43]:
parser2 = JsonOutputParser(pydantic_object=PersonInfo)
print(parser2.get_format_instructions())

STRICT OUTPUT FORMAT:
- Return only the JSON value that conforms to the schema. Do not include any additional text, explanations, headings, or separators.
- Do not wrap the JSON in Markdown or code fences (no ``` or ```json).
- Do not prepend or append any text (e.g., do not write "Here is the JSON:").
- The response must be a single top-level JSON value exactly as required by the schema (object/array/etc.), with no trailing commas or comments.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]} the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema (shown in a code block for readability only — do not include any backticks or Markdown in your output):


In [38]:
from langchain_core.output_parsers import PydanticOutputParser
parser = PydanticOutputParser(pydantic_object=PersonInfo)

prompt = ChatPromptTemplate(
    [
        {"role":"system", "content":"Output Format: {output_format}"},
        {"role":"user", "content":"{query}"}
    ],
    partial_variables={"output_format":parser.get_format_instructions()}
)
model = ChatOpenAI(model="gpt-5-mini")

query = prompt.invoke({"query":"이순신 장군에 대해서 알려줘."})
res = model.invoke(query)

In [39]:
res

AIMessage(content='{\n  "name": "이순신",\n  "yob": 1545,\n  "yod": 1598,\n  "profile": "조선 중기의 무신이자 해군 장군. 임진왜란(1592–1598) 동안 활약하며 한산도 대첩(1592), 명량 해전(1597) 등에서 결정적 승리를 거두어 일본의 침략을 방어하는 데 큰 공헌을 했다. 거북선 등의 전술적 도입과 기동성을 활용해 수적 열세를 극복했고, 난중일기(難中日記)를 남겨 전쟁 기간의 기록과 리더십을 전한다. 1598년 노량해전에서 전사했으며, 사후 충무공(忠武公)으로 추숭되었다."\n}', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 641, 'prompt_tokens': 274, 'total_tokens': 915, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 448, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Cncd19OfRisfZlFrkUJTgBTZDnmDN', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b2a61-8db6-7c11-b597-4529d3c0ae42-0', usage_metadata={'input_tokens': 274, 'output_tokens': 641, 'total_to

In [40]:
final_res = parser.invoke(res)
type(final_res)

__main__.PersonInfo

In [41]:
final_res

PersonInfo(name='이순신', yob=1545, yod=1598, profile='조선 중기의 무신이자 해군 장군. 임진왜란(1592–1598) 동안 활약하며 한산도 대첩(1592), 명량 해전(1597) 등에서 결정적 승리를 거두어 일본의 침략을 방어하는 데 큰 공헌을 했다. 거북선 등의 전술적 도입과 기동성을 활용해 수적 열세를 극복했고, 난중일기(難中日記)를 남겨 전쟁 기간의 기록과 리더십을 전한다. 1598년 노량해전에서 전사했으며, 사후 충무공(忠武公)으로 추숭되었다.')

In [42]:
final_res.profile

'조선 중기의 무신이자 해군 장군. 임진왜란(1592–1598) 동안 활약하며 한산도 대첩(1592), 명량 해전(1597) 등에서 결정적 승리를 거두어 일본의 침략을 방어하는 데 큰 공헌을 했다. 거북선 등의 전술적 도입과 기동성을 활용해 수적 열세를 극복했고, 난중일기(難中日記)를 남겨 전쟁 기간의 기록과 리더십을 전한다. 1598년 노량해전에서 전사했으며, 사후 충무공(忠武公)으로 추숭되었다.'

# LLM모델에 출력 형식을 설정

- ChatModel객체의 `with_structured_output(pydantic.BaseModel)` 을 이용해 모델의 출력 형식을 모델 자체에 추가할 수있다.
- `OutputParser`는 모델의 출력 결과를 받아서 형식을 변경해 준다. 그래서 Chain에 탈/부착을 통해 형식을 적용하거나 적용하지 않는 것을 자유롭게 할 수있다.
- 모델의 출력 결과를 항상 일정하게 할 경우에는 아예 **모델에 출력 형식을 설정할 수 있다.**

In [45]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

# 출력 schema 정의
class OutputSchema(BaseModel):
    is_real_person: bool = Field(..., description="요청한 인물이 실존임물인지 여부. True: 실존인물, False:실존인물이 아님")
    description: str = Field(..., description="요청한 인물에 대한 설명")

model = ChatOpenAI(model='gpt-5-mini')
res = model.invoke("이순신 장군에 대해 설명해줘.")

In [46]:
print(type(res))
print(res.content)

<class 'langchain_core.messages.ai.AIMessage'>
이순신(李舜臣)은 조선 중기의 무신이자 해군 장군으로, 임진왜란(1592–1598) 때 일본 수군을 상대로 탁월한 지휘와 전술로 큰 승리를 거둔 한국의 국민적 영웅입니다. 1545년에 태어나 1598년 노량해전에서 전사할 때까지 해상 방어를 책임졌고, 임진왜란 기간에 적어도 주요 해전에서 패하지 않은 것으로 널리 알려져 있습니다.

주요 업적과 사건
- 한산도 대첩(1592): 한산도 해전에서 학익진(학 날개 모양) 전술을 사용해 일본 연합 함대를 크게 격파, 남해 통제의 기반을 마련했습니다.
- 명량해전(1597): 명량 해협의 급격한 조류와 지형을 이용해 13척의 배로 수백 척의 적을 상대해 대승을 거둔 기념비적 전투입니다.
- 노량해전(1598): 퇴각하는 일본군을 추격하던 중 전사했으며, 전사 당시에도 전투 지휘를 계속했다는 기록이 전해집니다.
- 거북선: 철갑선(덮개와 창살, 포문을 갖춘 배)로 알려진 거북선의 운용과 화력 중심의 근대적 해전 운영에 기여한 것으로 평가됩니다.

인품과 기록
- 그는 강한 책임감, 엄격한 규율, 병사들에 대한 세심한 배려로 유명합니다. 개인 일기인 난중일기(亂中日記)는 전쟁 중의 전략·전술·심리·정치 상황을 기록한 중요한 역사 자료입니다.
- 정치적 반대 때문에 한때 파면·투옥되기도 했으나 백성과 실전에서의 능력으로 재기하여 전공을 세웠습니다.

평가와 유산
- 충무공(忠武公) 이순신이라는 존호가 내려졌고, 한국에서는 최고의 군사 영웅 중 한 사람으로 추앙됩니다. 기념비·동상·문학·영화(예: 영화 <명량>) 등으로 널리 기념되고 있습니다.
- 난중일기와 전적들은 해전 전술 연구 및 리더십 연구의 중요한 자료로 활용됩니다.

원하시면 각 전투의 전술 분석, 난중일기의 주요 내용, 또는 거북선의 설계와 운용 등 특정 주제에 대해 더 자세히 설명해 드리겠습니다.


In [47]:
# 모델에 응답 스키마를 추가
model_with_output = model.with_structured_output(OutputSchema)

res2 = model_with_output.invoke("이순신 장군에 대해 설명해줘.")

In [50]:
print(type(res2))
print(res2)
print(res2.is_real_person)

<class '__main__.OutputSchema'>
is_real_person=True description='이순신(1545–1598)은 조선 시대의 해군 장수로, 임진왜란(1592–1598) 기간에 일본 수군을 상대로 결정적 승리를 거둔 한국의 대표적 군인이자 민족적 영웅입니다. 한산도 대첩(1592) 등에서 학익진 등 혁신적 전술을 사용해 큰 공을 세웠고, 명량해전(1597)에서는 13척의 배로 수백 척의 적을 격파하는 전과를 올렸습니다. 거북선(거북선)의 운용과 해전 전략으로 유명하며, 난중일기라는 전쟁 일기로 당시의 기록을 남겼습니다. 노량해전(1598)에서 전사했으며, 사후 충무공(忠武公)이라는 시호를 받았고 오늘날까지 한국에서 가장 존경받는 역사적 인물 중 하나입니다.'
True


In [ ]:
res3 = model_with_output.invoke("홍길동에 대해 설명해줘.")
#모델 + OutputSchema 를 붙여서 물어보는 것과 같은 형식

In [52]:
res3

OutputSchema(is_real_person=False, description='홍길동은 조선 시대를 배경으로 한 한글 소설 『홍길동전』의 주인공으로, 일반적으로 작가 허균(許筠, 1569–1618)에게 귀속되지만 저자와 창작 시기에는 논쟁이 있다. 홍길동은 서얼(첩의 아들)로 태어나 신분 차별과 부패한 양반 사회에 맞서 의적 활동을 펼치고, 부당한 권력에 맞서 백성을 돕는 이야기의 주인공이다. 도적떼를 이끌고 불의를 바로잡고 결국 새로운 공동체(또는 타국의 왕)이 되어 정의로운 나라를 세운다는 내용이 중심 주제이며, 신분제와 사회적 불평등에 대한 비판과 민중적 영웅상을 보여준다. 한국 문학과 대중문화에서 ‘의적’ 또는 ‘사회적 약자를 돕는 영웅’의 전형으로 널리 알려져 있고, 여러 영화·드라마·만화 등으로 재창조되어 왔다.')

In [53]:
model.invoke("홍길동에 대해 설명해 줘.")

AIMessage(content='홍길동은 조선 후기의 소설 홍길동전(洪吉童傳)의 주인공으로, 일반적으로 허균(許筠, 1569–1618)에게서 비롯된 작품으로 알려져 있습니다. 대중적으로는 ‘한국의 로빈 후드’로 불리며 신분제·사회 부조리에 맞서는 영웅으로 널리 사랑받습니다.\n\n핵심 내용(요약)\n- 출생과 신분 문제: 홍길동은 양반의 서자로 태어나 뛰어난 재능과 지식을 가졌지만, 서얼(庶孼) 신분 때문에 과거 응시나 관직 진출에서 배제됩니다.\n- 사회적 갈등과 탈주: 신분 차별과 부패한 권력에 분노한 그는 집을 떠나 산적 무리를 이끌게 되고, 도적질로 얻은 재물을 부당하게 착취하는 양반·관리들로부터 약자들을 보호하는 데 씁니다.\n- 초자연적 능력과 활약: 작품에는 홍길동의 변신·무술 같은 초능력적 요소도 등장하여 적들을 물리치고 의적 행각을 펼칩니다.\n- 결말의 여러 변형: 전승과 판본에 따라 결말이 다르지만, 많은 판본에서 그는 결국 새로운 나라를 세우거나 외국에서 높은 지위에 오르는 등 ‘정의로운 통치자’가 되는 식으로 끝납니다.\n\n주제와 의의\n- 신분제·부패 비판: 양반 중심의 신분제와 관료사회, 부패한 제도에 대한 날카로운 풍자와 비판이 중심 주제입니다.\n- 개인의 자유와 정의: 혈통·신분보다 능력과 정의가 우선해야 한다는 메시지를 담고 있습니다.\n- 문학사적 위치: 홍길동전은 한글 소설의 대표작 중 하나로 평가되며, 근대적 소설의 전형적 요소(개인 서사·사회 비판 등)를 갖춘 작품으로 중요한 문학사적 가치를 지닙니다.\n\n현대적 영향\n- 수많은 영화·드라마·연극·만화·뮤지컬로 각색되었고, ‘홍길동’은 한국에서 익명인물(예: 서류상의 예시 이름)이나 의적·영웅의 전형으로 자주 언급됩니다.\n- 사회 정의나 신분 문제를 논할 때 자주 인용되는 문화적 아이콘입니다.\n\n더 원하시면:\n- 작품의 상세 줄거리(장별 요약)\n- 원문에서 나타나는 특정 장면·인물 분석\n- 허균의 생애와 작품 세계 비교\n- 현대 작품(드라마·영화)별 

# Streaming 방식 응답 처리

- Streaming 방식 응답 처리란, LLM이 텍스트를 모두 생성할 때까지 기다리지 않고, 생성되는 즉시 **부분적인 결과**를 실시간으로 전달받아 처리하는 방식을 의미한다. 이는 사용자가 응답을 더 빠르게 인지할 수 있게 해 주며, 특히 대화형 서비스, 실시간 UI 출력, 긴 문서 생성과 같은 상황에서 매우 유용하게 활용된다.

- `invoke()` 요청으로 받는 응답은 **비 스트리밍 방식**으로 모든 응답 텍스트 생성이 완료된 이후 그 결과를 한 번에 반환하는 구조이다. 반면 Streaming 방식은 **토큰(token) 단위 또는 여러 토큰이 묶인 청크(chunk) 단위**로 연속적인 데이터 스트림을 전송한다는 점에서 큰 차이가 있다. 즉, Streaming은 마치 사람이 타이핑을 치듯이 응답을 실시간으로 “흘려보내는” 방식이라고 이해할 수 있다.

- `모델.invoke(input, config)` → 응답 데이터
    - 모델이 전체 응답을 모두 생성한 뒤, 최종 결과를 한 번에 반환하는 방식이다.
    - 배치 처리나 후처리가 중요한 경우에 적합하다.
- `모델.stream(input, config)` → Iterator
    - 모델이 토큰을 생성하는 즉시, 순차적으로 결과를 제공하는 Iterator 형태로 반환한다.
    - 실시간 출력, 대화형 인터페이스, 웹 스트리밍 등에 특히 적합하다.

In [54]:
model = ChatOpenAI(model="gpt-5-mini") # 이전 버전에서는 streaming=Ture 주어야함

iterator = model.stream("서울의 유명한 관광지를 소개해 줘. 추천 이유도 알려줘.")
print(type(iterator))
for token in iterator:
    print(token)

<class 'generator'>
content='' additional_kwargs={} response_metadata={'model_provider': 'openai'} id='lc_run--019b2b41-fb50-77b2-b5e6-024bb4690ed7'
content='아' additional_kwargs={} response_metadata={'model_provider': 'openai'} id='lc_run--019b2b41-fb50-77b2-b5e6-024bb4690ed7'
content='래' additional_kwargs={} response_metadata={'model_provider': 'openai'} id='lc_run--019b2b41-fb50-77b2-b5e6-024bb4690ed7'
content='는' additional_kwargs={} response_metadata={'model_provider': 'openai'} id='lc_run--019b2b41-fb50-77b2-b5e6-024bb4690ed7'
content=' 서울' additional_kwargs={} response_metadata={'model_provider': 'openai'} id='lc_run--019b2b41-fb50-77b2-b5e6-024bb4690ed7'
content='의' additional_kwargs={} response_metadata={'model_provider': 'openai'} id='lc_run--019b2b41-fb50-77b2-b5e6-024bb4690ed7'
content=' 대표' additional_kwargs={} response_metadata={'model_provider': 'openai'} id='lc_run--019b2b41-fb50-77b2-b5e6-024bb4690ed7'
content='적인' additional_kwargs={} response_metadata={'model_provide

In [56]:
for token in model.stream("서울의 유명한 관광지를 소개해 줘. 추천 이유도 알려줘."):
    print(token.content, end="")

서울에서 꼭 가볼 만한 유명 관광지들을 추천 이유와 함께 정리해 드릴게요. 원하시면 일정(1일/2일/3일)도 맞춰 드립니다.

- 경복궁  
  - 왜: 조선의 대표 고궁으로 근정전·경회루 등 장대한 전통 건축을 볼 수 있고, 수문장 교대식과 한복 체험으로 사진 찍기 좋습니다.  
  - 팁: 한복 착용 시 입장료 면제, 오전에 비교적 한가함.

- 창덕궁(후원)  
  - 왜: 자연과 어우러진 후원(비원)이 유명한 유네스코 세계문화유산으로 가이드 투어로 더 깊이 즐길 수 있습니다.  
  - 팁: 후원은 시간대별 예약 투어 권장.

- 북촌한옥마을  
  - 왜: 잘 보존된 전통 한옥 골목을 걸으며 옛 서울 분위기를 느낄 수 있고, 사진 스팟이 많습니다.  
  - 팁: 골목이 좁아 이른 시간이나 평일 방문 추천.

- 인사동  
  - 왜: 전통 공예품, 갤러리, 찻집과 한식당이 모여 있어 한옥·한국 문화상품을 구경·구입하기 좋습니다.  
  - 팁: 주말에는 인사동 거리공연과 붐빔.

- 명동  
  - 왜: 서울 대표 쇼핑·스트리트푸드 지역으로 화장품·패션 쇼핑과 길거리 음식 체험에 최적입니다.  
  - 팁: 밤에 더 활기차고 외국인 관광객에게 인기.

- 남대문·동대문 시장  
  - 왜: 남대문은 전통시장(의류·기념품), 동대문은 패션 도매·쇼핑타운과 디자인플라자(DDP)로 쇼핑 취향별 선택 가능.  
  - 팁: 동대문은 야간 쇼핑과 DDP 야경이 매력적.

- 광장시장·광장시장 먹거리  
  - 왜: 빈대떡, 마약김밥, 육회 등 전통 시장 음식으로 현지 식문화를 체험하기 좋습니다.  
  - 팁: 소량으로 여러 메뉴를 맛보세요.

- 남산서울타워(N서울타워)  
  - 왜: 서울 시내 전경을 한눈에 보이는 전망 명소로 야경과 사랑의 자물쇠로 유명합니다.  
  - 팁: 케이블카 또는 등산로 이용 가능, 해질녘 추천.

- 한강공원(여의도·반포 등)  
  - 왜: 피크닉, 자전거, 유람선, 반포대교 달빛무지개분수 등 야외 활동과 휴식에 최적입니

KeyboardInterrupt: 

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-5-nano"
)
#질의
response=model.invoke("Langchain은 어떤 Framework인지 설명해줘.")

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-5-nano",
    streaming=True
)
#질의
res=model.stream("Langchain은 어떤 Framework인지 설명해줘.")
for token in res:
    print(token.content, end="")